In [9]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,label,news
0,False,Says the Annies List political group supports ...
1,True,When did the decline of coal start? It started...
2,True,"Hillary Clinton agrees with John McCain ""by vo..."
3,False,Health care reform legislation is likely to ma...
4,True,The economic turnaround started at the end of ...


In [10]:
###Drop Nan Values
df=df.dropna()
## Get the Independent Features

X=df.drop('label',axis=1)

In [11]:
y=df['label']
X.shape

(10240, 1)

In [12]:
y.shape

(10240,)

In [13]:
import tensorflow as tf
tf.__version__


'2.20.0-rc0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
voc_size=5000

In [16]:
messages=X.copy()
print(messages)

                                                    news
0      Says the Annies List political group supports ...
1      When did the decline of coal start? It started...
2      Hillary Clinton agrees with John McCain "by vo...
3      Health care reform legislation is likely to ma...
4      The economic turnaround started at the end of ...
...                                                  ...
10235  There are a larger number of shark attacks in ...
10236  Democrats have now become the party of the [At...
10237  Says an alternative to Social Security that op...
10238  On lifting the U.S. Cuban embargo and allowing...
10239  The Department of Veterans Affairs has a manua...

[10240 rows x 1 columns]


In [28]:
# print(messages[1])

In [ ]:
messages = df[['news']]  # double brackets to keep it as DataFrame
messages.reset_index(inplace=True)

In [18]:
messages['news'][1]

'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.'

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Sakshath
[nltk_data]     Shetty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['news'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
from tensorflow.keras.preprocessing.text import one_hot
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [32]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4737 1475 2311]
 [   0    0    0 ...  861 4140 2934]
 [   0    0    0 ... 3614 4388 3657]
 ...
 [   0    0    0 ... 3183 1202 2526]
 [   0    0    0 ... 3732 2551  273]
 [   0    0    0 ... 1840 3691 4473]]


In [26]:
embedded_docs[0]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2936,
       2903, 2698, 3862, 1068,  885, 2341, 4737, 1475, 2311], dtype=int32)

In [27]:

embedding_vector_features=40
model=Sequential()
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features, input_shape=(sent_length,)))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build(input_shape=(None, sent_length))
print(model.summary())

E:\work\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [33]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [34]:
import numpy as np

X_train = np.array(embedded_docs)  # processed training titles
y_train = np.array(y)         # corresponding labels (0/1)


In [35]:
X_final.shape,y_final.shape

((10240, 20), (10240,))

In [41]:
# Load test dataset
test_df = pd.read_csv("train_lower.csv")
test_df = test_df.dropna()

X_test = test_df['news']
y_test = test_df['label']

# Preprocess test dataset
corpus_test = []
for i in range(len(X_test)):
    review = re.sub('[^a-zA-Z]', ' ', X_test.iloc[i])
    review = review.lower().split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus_test.append(review)

onehot_test = [one_hot(words, voc_size) for words in corpus_test]
embedded_test = pad_sequences(onehot_test, padding='pre', maxlen=sent_length)

X_test_final = np.array(embedded_test)
y_test_final = np.array(y_test)

# Train model on training dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate on separate test dataset
loss, accuracy = model.evaluate(X_test_final, y_test_final)
print("Test Accuracy:", accuracy)


Epoch 1/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9884 - loss: 0.0347 - val_accuracy: 0.5493 - val_loss: 3.6861
Epoch 2/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9889 - loss: 0.0352 - val_accuracy: 0.5469 - val_loss: 3.6149
Epoch 3/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9883 - loss: 0.0375 - val_accuracy: 0.5396 - val_loss: 3.5059
Epoch 4/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9890 - loss: 0.0354 - val_accuracy: 0.5464 - val_loss: 3.6241
Epoch 5/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9924 - loss: 0.0243 - val_accuracy: 0.5483 - val_loss: 3.7330
Epoch 6/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9932 - loss: 0.0208 - val_accuracy: 0.5479 - val_loss: 3.7580
Epoch 7/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9954 - loss: 0.0143 - val_accuracy: 0.5518 - val_loss: 4.1699
Epoch 8/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9954 - loss: 0.0150 - val_accuracy: 0.